Notes

In [ ]:
import sympy as sp
import matplotlib.pyplot as plt
%matplotlib inline


# initiate pretty print
sp.init_printing()

# define parameter
t = sp.Symbol('t')

Define polynomial

In [ ]:
my_polynomial = t**2 + 1
my_polynomial

Plot the polynomial and find roots (solve p for t), including multiplicities

In [ ]:
#ax = sp.plot(my_polynomial, xlim=(-2, 2), ylim=(0,4), aspect_ratio=(1,1))

roots = sp.roots(my_polynomial)
roots

Hankel matrix of polynomial

In [ ]:
p = sp.Poly(my_polynomial, t, domain='RR')
coeffs = p.all_coeffs()

deg = p.degree()

n = sp.symbols('n')
newton_sums = [deg, -1*coeffs[1], coeffs[1]**2 - 2*coeffs[2]]
# https://brilliant.org/wiki/newtons-identities/

# Construct the Hankel matrix
hankel_matrix = sp.zeros(deg)
for i in range(deg):
    for j in range(deg):
        hankel_matrix[i, j] = newton_sums[i + j]

hankel_matrix